In [ ]:
# 从原始 OSM 区域中获取系统的元素样本
# -*- coding: utf-8 -*-
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "C:/Users/lk235/Desktop/Project_1/wenzhou.osm"  # Replace this with your osm file
SAMPLE_FILE = "C:/Users/lk235/Desktop/Project_1/sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')
    print 'WRITE END'

WRITE END


In [2]:
# 在地图中的每个类别有多少数据
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict
import re
import os

SAMPLE_FILE = "C:/Users/lk235/Desktop/Project_1/sample.osm"


       
def count_tags(filename):
    osm_file = open(filename, "r")
    tags = {}
    pattern = re.compile("'(.*)'")
    for event, elem in ET.iterparse(osm_file):
        k = str(pattern.findall(str(elem))).strip('[]').strip('\'')
        # k = pattern.findall(str(elem))
        
        if k not in tags.keys():
            tags[k] = 1
        else:
            tags[k] = tags[k] + 1
        
    return tags
print 'start'
print count_tags('sample.osm')
print os.linesep


start


{'node': 42224, 'nd': 49410, 'member': 350, 'tag': 8129, 'relation': 18, 'way': 2749, 'osm': 1}




In [11]:
# 审查路名：way节点下发现名字中有非中文、乱码等情况、非温州道路;如果英文路名存在中文字段，则用中文字段替换
# node节点下名字非中文，uu等无法理解的名字；china-class字段全部为英文或拼音，应统一改成中文
# 审查代码如下：
import re
from collections import defaultdict
import xml.etree.cElementTree as ET
import sys
reload(sys)
sys.setdefaultencoding('utf8')

# SAMPLE_FILE = "C:/Users/lk235/Desktop/Project_1/sample.osm"
# def is_street_name(elem):
#     return (elem.attrib['k'] == "addr:street")
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
# G104_re = re.compile(r'104')
G104_none_line_char_re = re.compile(ur'\u56fd\u9053(?!\u7ebf)')
G104_none_G_char_re = re.compile(r'^(?!G)104')
S41_re = re.compile(r'S41')
S233_re = re.compile(ur'223\u7701\u9053')
S325_re = re.compile(r'S325')
S1_re = re.compile(r'S1')
X9_re = re.compile(r'X[0-9]{3}')
chinese_re = re.compile(ur'[\u4e00-\u9fff]')
road_have_number_re = re.compile(r'\d')



mapping = { 'S41': 'S223省道',
            '223省道': 'S223省道',
            'S325': 'S325省道',
            'S1': 'S1市域铁路',
            "NationalRoad104": "G104国道线",
            "104NationalRoad": "G104国道线",
            '国道':'国道线',
            '104':"G104"
            
            }

def update_name(name):
    S41 = S41_re.search(name)
    S233 = S233_re.search(name)
    S325 = S325_re.search(name)
    S1 = S1_re.search(name)
    X9 = X9_re.search(name)
   
    if name == 'NationalRoad104':
        name = mapping['NationalRoad104']
    elif name == '104NationalRoad':
        name = mapping['104NationalRoad']
    elif S41:
        old_name =  str(S41.group())
        replace = mapping[old_name]
        name = name.replace(old_name,replace)
    elif S233:
        old_name =  str(S233.group())
        replace = mapping[old_name]
        name = name.replace(old_name,replace)
    elif S325:
        old_name =  str(S325.group())
        replace = mapping[old_name]
        name = name.replace(old_name,replace)
    elif S1:
        old_name =  str(S1.group())
        replace = mapping[old_name]
        name = name.replace(old_name,replace)
    elif X9:
        old_name = str(X9.group())
        replace = old_name + '县道'
        name = name.replace(old_name,replace)
    else:
        G104_none_line_char = G104_none_line_char_re.search(name)
        G104_none_G_char = G104_none_G_char_re.search(name)
        if G104_none_line_char:
            old_name =  str(G104_none_line_char.group())
            replace = mapping[old_name]
            name = name.replace(old_name,replace)
        if G104_none_G_char:
            old_name =  str(G104_none_G_char.group())
            replace = mapping[old_name]
            name = name.replace(old_name,replace)

    return name

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    count = 0
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        # if elem.tag == "node" or elem.tag == "way":
        if elem.tag == "way":
            for tag in elem.iter("tag"):
                # print tag.attrib['k']
                if tag.attrib['k'] == 'name:zh':
                    road_name = tag.attrib['v'].replace(' ','')
                    # road_name = tag.attrib['v']
                    if not problemchars.search(road_name):
                        # if not chinese_re.search(road_name):
                        # if road_have_number_re.search(road_name):
                            # road_name = update_name(road_name)
                        
                            count = count + 1
                            print road_name
                    # if road_have_number_re.search(road_name):
                    #     road_name = update_name(road_name)
                        
                    
                        
                # if is_street_name(tag):
              
                    # audit_street_type(street_types, tag.attrib['v'])
    print count               
    osm_file.close()
    return street_types

audit('C:/Users/lk235/Desktop/Project_1/wenzhou.osm')
# audit('sample.osm')

# S1 = '104 国道'
# print S1.replace(' ','')
# print unicode('国道','utf8')
print 'END'
# s1 = {'国道':104,'S':10}
# print s1['S']
# s1['国道'] = '国道线'
# print s1['国道']




In [17]:
# 准备数据

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "C:/Users/Administrator/Desktop/Project_1"
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
G104_none_line_char_re = re.compile(ur'\u56fd\u9053(?!\u7ebf)')
G104_none_G_char_re = re.compile(r'^(?!G)104')
S41_re = re.compile(r'S41')
S233_re = re.compile(ur'223\u7701\u9053')
S325_re = re.compile(r'S325')
S1_re = re.compile(r'S1')
X9_re = re.compile(r'X[0-9]{3}')
chinese_re = re.compile(ur'[\u4e00-\u9fff]')
road_have_number_re = re.compile(r'\d')
Is_road_re = re.compile(ur'[\u8def|\u7ebf|\u9053]')
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

# SCHEMA = schema.schema

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

mapping = { 'S41': 'S223省道',
            '223省道': 'S223省道',
            'S325': 'S325省道',
            'S1': 'S1市域铁路',
            "NationalRoad104": "G104国道线",
            "104NationalRoad": "G104国道线",
            '国道':'国道线',
            '104':"G104"
            
            }

def update_name(name):
    S41 = S41_re.search(name)
    S233 = S233_re.search(name)
    S325 = S325_re.search(name)
    S1 = S1_re.search(name)
    X9 = X9_re.search(name)
   
    if name == 'NationalRoad104':
        name = mapping['NationalRoad104']
    elif name == '104NationalRoad':
        name = mapping['104NationalRoad']
    elif S41:
        old_name =  str(S41.group())
        replace = mapping[old_name]
        name = name.replace(old_name,replace)
    elif S233:
        old_name =  str(S233.group())
        replace = mapping[old_name]
        name = name.replace(old_name,replace)
    elif S325:
        old_name =  str(S325.group())
        replace = mapping[old_name]
        name = name.replace(old_name,replace)
    elif S1:
        old_name =  str(S1.group())
        replace = mapping[old_name]
        name = name.replace(old_name,replace)
    elif X9:
        old_name = str(X9.group())
        replace = old_name + '县道'
        name = name.replace(old_name,replace)
    else:
        G104_none_line_char = G104_none_line_char_re.search(name)
        G104_none_G_char = G104_none_G_char_re.search(name)
        if G104_none_line_char:
            old_name =  str(G104_none_line_char.group())
            replace = mapping[old_name]
            name = name.replace(old_name,replace)
        if G104_none_G_char:
            old_name =  str(G104_none_G_char.group())
            replace = mapping[old_name]
            name = name.replace(old_name,replace)

    return name

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    

    if element.tag == 'node':
        node_attribs = element.attrib
        el_id = element.attrib['id']
        for k in node_attribs.keys():
            if k not in node_attr_fields:
                del node_attribs[k]
 
        for tag in element.iter("tag"):
            
            key = tag.attrib['k']
            value = tag.attrib['v'].replace(' ','')
            
            if not problemchars.search(value):
                
                if key == 'name' or key == 'name.zh':
                    if chinese_re.search(tag.attrib['v']):
                        tag_dict = {}
                        tag_dict['type'] = default_tag_type
                        tag_dict['key'] = key
                        tag_dict['id'] = el_id   
                        tag_dict['value'] = value
                        tags.append(tag_dict)
                            
                else: 
                     tag_dict = {}
                     tag_dict['type'] = default_tag_type
                     tag_dict['key'] = key
                     tag_dict['id'] = el_id   
                     tag_dict['value'] = value
                     tags.append(tag_dict)
        # print {'node': node_attribs, 'node_tags': tags}
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        way_attribs = element.attrib
        el_id = element.attrib['id']
        for k in way_attribs.keys():
            if k not in way_attr_fields:
                del way_attribs[k]
       
        position = 0
        for tag in element.iter():
            
            
            
            if tag.tag == 'tag':
                 key = tag.attrib['k']
                 value = tag.attrib['v'].replace(' ','')

                 if not problemchars.search(value):
                     
                     if key == 'name'or key =='name:zh':
                         value = update_name(value)
                         if chinese_re.search(value):
                             tag_dict = {}
                             if Is_road_re.search(tag.attrib['v']):
                                 tag_dict['type'] = 'Road'
                             else:
                                 tag_dict['type'] = default_tag_type
                             tag_dict['id'] = el_id
                             tag_dict['key'] = key
                             tag_dict['value'] = value
                             tags.append(tag_dict)
                                 
                     else:
                         tag_dict = {}
                         tag_dict['value'] = value
                         tag_dict['type'] = default_tag_type
                         tag_dict['key'] = key
                         tag_dict['id'] = el_id
                         tags.append(tag_dict)

                     
                
            if tag.tag == 'nd':
                way_dict = {}
                way_dict['id'] = el_id
                way_dict['node_id'] = tag.attrib['ref']
                way_dict['position'] = position
                position = position + 1
                way_nodes.append(way_dict)
        # print  {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}  
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
    
def get_element(osm_file, tags=('node', 'way')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            
# def validate_element(element, validator, schema=SCHEMA):
#     """Raise ValidationError if element does not match schema"""
#     if validator.validate(element, schema) is not True:
#         field, errors = next(validator.errors.iteritems())
#         message_string = "\nElement of type '{0}' has the following errors:\n{1}"
#         error_string = pprint.pformat(errors)
#         
#         raise Exception(message_string.format(field, error_string))
    
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
            

def process_map(file_in):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        # validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                # if validate is True:
                #     validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


# elements = get_element('C:/Users/Administrator/Desktop/Project_1/wenzhou.osm')
# for element in elements:
#     shape_element(element)

process_map('C:/Users/lk235/Desktop/Project_1/wenzhou.osm')
print 'END'